In [1]:
%load_ext bigdata
%hive_start
%timeout 300

In [2]:
!hadoop fs -rm -f -r tmp;

Deleted output


In [3]:
%%hive
DROP TABLE IF EXISTS data;
CREATE TABLE data (letra STRING,
                   fecha STRING,
                   num INT)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\t';

DROP TABLE IF EXISTS data;
OK
Time taken: 13.913 seconds
CREATE TABLE data (letra STRING,
                   fecha STRING,
                   num INT)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\t';
OK
Time taken: 1.031 seconds


In [4]:
%%hive
LOAD DATA LOCAL INPATH "data.tsv" OVERWRITE INTO TABLE data;
--SELECT * FROM data LIMIT 5;

LOAD DATA LOCAL INPATH "data.tsv" OVERWRITE INTO TABLE data;
Loading data to table default.data
OK
Time taken: 1.351 seconds
--SELECT * FROM data LIMIT 5;


In [5]:
%%hive
DROP TABLE IF EXISTS salida;
CREATE TABLE salida
AS
    SELECT letra, count(1) AS cuenta
    FROM
        data
GROUP BY
    letra
ORDER BY
    letra;
    
--SELECT * 
--FROM salida 
--LIMIT 5;     

DROP TABLE IF EXISTS salida;
OK
Time taken: 0.149 seconds
CREATE TABLE salida
AS
    SELECT letra, count(1) AS cuenta
    FROM
        data
GROUP BY
    letra
ORDER BY
    letra;
Query ID = root_20200112002056_f7596090-0f9b-403b-84fc-031185716960
Total jobs = 2
Launching Job 1 out of 2
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1578776258951_0009, Tracking URL = http://e19ecdc0743b:8088/proxy/application_1578776258951_0009/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1578776258951_0009
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2020-01-12 00:21:08,238 Stage-1 map = 0%,  reduce = 0%
2020-01-12 00:21:16,484 S

In [6]:
%%hive
INSERT OVERWRITE DIRECTORY 'tmp/000000_0'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT
    *
FROM
    salida;

INSERT OVERWRITE DIRECTORY 'tmp/output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT
    *
FROM
    salida;
Query ID = root_20200112002249_5bac64bc-0e59-4658-b76c-51c71e70e598
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1578776258951_0011, Tracking URL = http://e19ecdc0743b:8088/proxy/application_1578776258951_0011/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1578776258951_0011
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 0
2020-01-12 00:22:57,148 Stage-1 map = 0%,  reduce = 0%
2020-01-12 00:23:02,628 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 0.95 sec
MapReduce Total cumulative CPU time: 950 msec
Ended Job = job_1578776258951_0011
Stage-3 is selected by condition resolver.
Stage-2 is filtered out by condition resolver.
Stage-4 is filtered out by condition resolver.
Moving data to directory hdfs://0.0.0.0:9000/user/root/tmp/output/.hive-stagin

In [11]:
!hadoop fs -copyToLocal ./tmp/000000_0 output

In [12]:
%hive_quit